In [ ]:
from minio import Minio
from minio.error import S3Error
import os
from dotenv import load_dotenv
# Cargar variables del archivo .env
load_dotenv()

In [ ]:
def conectar_minio(endpoint='localhost:9000', 
                  access_key=None, 
                  secret_key=None, 
                  secure=False):
    """
    Establece una conexión con un servidor MinIO.
    
    Parámetros:
    -----------
    endpoint : str, opcional
        Dirección del servidor MinIO (por defecto 'localhost:9000')
    access_key : str, requerido
        Usuario/access key para la autenticación
    secret_key : str, requerido
        Contraseña/secret key para la autenticación
    secure : bool, opcional
        Si es True, usa HTTPS. Si es False, usa HTTP (por defecto False)
    
    Retorna:
    --------
    Minio
        Cliente MinIO conectado
    
    Excepciones:
    ------------
    ValueError
        Si no se proporcionan access_key o secret_key
    S3Error
        Si hay un error al conectar con el servidor MinIO
    """
    
    if not access_key or not secret_key:
        raise ValueError("Se requieren access_key y secret_key para la conexión")
    
    try:
        # Crear cliente MinIO
        cliente = Minio(
            endpoint,
            access_key=access_key,
            secret_key=secret_key,
            secure=secure
        )
        
        # Verificar conexión listando los buckets (opcional)
        buckets = cliente.list_buckets()
        print(f"Conexión exitosa a MinIO en {endpoint}")
        print(f"Buckets disponibles: {[bucket.name for bucket in buckets]}")
        
        return cliente
    
    except S3Error as err:
        print(f"Error al conectar con MinIO: {err}")
        raise

Subir archivos minio

In [ ]:
from io import BytesIO, StringIO
import pandas as pd
from minio.error import S3Error  # Asegúrate de importar S3Error

def guardar_df_en_minio(minio_client, df, bucket_name, ruta_destino, 
                        formato='parquet', crear_bucket=False):
    """
    Guarda un DataFrame directamente en un bucket de MinIO.
    """
    
    if not minio_client:
        raise ValueError("Se requiere un cliente MinIO válido")
    if not isinstance(df, pd.DataFrame):
        raise ValueError("El parámetro df debe ser un pandas.DataFrame")
    if not bucket_name or not ruta_destino:
        raise ValueError("bucket_name y ruta_destino son requeridos")
    
    formatos_soportados = {
        'parquet': {
            'mime': 'application/parquet',
            'writer': lambda buffer: df.to_parquet(buffer, index=False),
            'extension': '.parquet',
            'buffer_type': BytesIO
        },
        'csv': {
            'mime': 'text/csv',
            'writer': lambda buffer: df.to_csv(buffer, index=False),
            'extension': '.csv',
            'buffer_type': StringIO
        },
        'json': {
            'mime': 'application/json',
            'writer': lambda buffer: df.to_json(buffer, orient='records'),
            'extension': '.json',
            'buffer_type': BytesIO
        },
        'excel': {
            'mime': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
            'writer': lambda buffer: df.to_excel(buffer, index=False),
            'extension': '.xlsx',
            'buffer_type': BytesIO
        }
    }
    
    formato = formato.lower()
    if formato not in formatos_soportados:
        raise ValueError(f"Formato '{formato}' no soportado. Use: {list(formatos_soportados.keys())}")
    
    if not ruta_destino.lower().endswith(formatos_soportados[formato]['extension']):
        ruta_destino += formatos_soportados[formato]['extension']
    
    try:
        if crear_bucket and not minio_client.bucket_exists(bucket_name):
            minio_client.make_bucket(bucket_name)
            print(f"Bucket '{bucket_name}' creado exitosamente")

        if not minio_client.bucket_exists(bucket_name):
            raise S3Error(f"El bucket '{bucket_name}' no existe", bucket_name, None, 404)
        
        # Crear buffer en memoria
        buffer = formatos_soportados[formato]['buffer_type']()
        formatos_soportados[formato]['writer'](buffer)

        # Ajuste para CSV (convertir a BytesIO)
        if formato == 'csv':
            buffer.seek(0)
            data = BytesIO(buffer.getvalue().encode('utf-8'))
            length = len(data.getvalue())
        else:
            buffer.seek(0)
            data = buffer
            length = buffer.getbuffer().nbytes
        
        # Subir a MinIO
        minio_client.put_object(
            bucket_name=bucket_name,
            object_name=ruta_destino,
            data=data,
            length=length,
            content_type=formatos_soportados[formato]['mime']
        )
        
        ruta_completa = f"{bucket_name}/{ruta_destino}"
        print(f"DataFrame guardado exitosamente en: {ruta_completa}")
        return ruta_completa

    except S3Error as err:
        print(f"Error al guardar DataFrame en MinIO: {err}")
        raise
    except Exception as e:
        print(f"Error al procesar el DataFrame: {e}")
        raise


Leer archivos Minio

In [ ]:
import PyPDF2
import docx

def extraer_archivo_minio(minio_client, bucket_name, ruta_archivo, tipo_archivo):
    """
    Extrae un archivo de MinIO y lo retorna en el formato adecuado.
    
    Parámetros:
    -----------
    minio_client : Minio
        Cliente MinIO ya conectado
    bucket_name : str
        Nombre del bucket donde está el archivo
    ruta_archivo : str
        Ruta completa del archivo dentro del bucket
    tipo_archivo : str
        Tipo de archivo a extraer ('pdf', 'word', 'excel', 'csv', 'parquet')
    
    Retorna:
    --------
    Depende del tipo de archivo:
    - 'pdf': Texto extraído (str)
    - 'word': Documento de python-docx
    - 'excel': DataFrame de pandas
    - 'csv': DataFrame de pandas
    - 'parquet': DataFrame de pandas
    
    Excepciones:
    ------------
    ValueError
        Si los parámetros son inválidos o el tipo no es soportado
    S3Error
        Si hay un error al acceder al archivo o el bucket no existe
    """
    
    # Validaciones iniciales
    if not minio_client:
        raise ValueError("Se requiere un cliente MinIO válido")
    if not bucket_name or not ruta_archivo or not tipo_archivo:
        raise ValueError("bucket_name, ruta_archivo y tipo_archivo son requeridos")
    
    tipo_archivo = tipo_archivo.lower()
    tipos_soportados = ['pdf', 'word', 'excel', 'csv', 'parquet']
    
    if tipo_archivo not in tipos_soportados:
        raise ValueError(f"Tipo de archivo '{tipo_archivo}' no soportado. Use: {tipos_soportados}")
    
    try:
        # Obtener el objeto de MinIO
        response = minio_client.get_object(bucket_name, ruta_archivo)
        data = BytesIO(response.read())
        data.seek(0)
        
        # Procesar según el tipo de archivo
        if tipo_archivo == 'pdf':
            # Extraer texto de PDF
            pdf_reader = PyPDF2.PdfReader(data)
            text = "\n".join([page.extract_text() for page in pdf_reader.pages])
            return text
        
        elif tipo_archivo == 'word':
            # Retornar documento de Word
            return docx.Document(data)
        
        elif tipo_archivo in ['excel', 'csv', 'parquet']:
            # Leer con pandas según el formato
            if tipo_archivo == 'excel':
                return pd.read_excel(data)
            elif tipo_archivo == 'csv':
                return pd.read_csv(data)
            elif tipo_archivo == 'parquet':
                return pd.read_parquet(data)
        
    except S3Error as err:
        print(f"Error al acceder al archivo en MinIO: {err}")
        raise
    except Exception as e:
        print(f"Error al procesar el archivo {ruta_archivo}: {e}")
        raise
    finally:
        response.close()
        response.release_conn()

conectarse base de datos

In [ ]:
import psycopg2
from psycopg2 import OperationalError

def conectar_postgres(db_name, host, user, password, port=5432):
    """
    Establece una conexión con una base de datos PostgreSQL.
    
    Parámetros:
    -----------
    db_name : str
        Nombre de la base de datos
    host : str
        Dirección del servidor (hostname o IP)
    user : str
        Nombre de usuario para la autenticación
    password : str
        Contraseña para la autenticación
    port : int, opcional
        Puerto de conexión (por defecto 5432)
    
    Retorna:
    --------
    psycopg2.extensions.connection
        Objeto de conexión a PostgreSQL
    
    Excepciones:
    ------------
    OperationalError
        Si hay un error al conectar con la base de datos
    ValueError
        Si los parámetros requeridos no son proporcionados
    """
    
    # Validar parámetros requeridos
    if not all([db_name, host, user, password]):
        raise ValueError("db_name, host, user y password son parámetros requeridos")
    
    try:
        # Establecer la conexión
        conexion = psycopg2.connect(
            dbname=db_name,
            host=host,
            user=user,
            password=password,
            port=port,
            client_encoding='UTF8'  # Forzar codificación UTF-8
        )
        
        # Verificar la conexión
        cursor = conexion.cursor()
        cursor.execute("SELECT version();")
        version = cursor.fetchone()
        cursor.close()
        
        print(f"Conexión exitosa a PostgreSQL {version[0]}")
        return conexion
    
    except OperationalError as err:
        print(f"Error al conectar a PostgreSQL: {err}")
        raise

In [ ]:
from datetime import datetime

def generar_ruta_fecha(separador='/', fecha=None):
    """
    Genera un string con la fecha en formato año/mes/día.
    
    Parámetros:
    -----------
    separador : str, opcional
        Carácter separador entre componentes (por defecto '/')
    fecha : datetime, opcional
        Fecha específica a formatear (si None, usa fecha actual)
    
    Retorna:
    --------
    str
        String con el formato 'YYYY{separador}MM{separador}DD'
    """
    # Usar fecha actual si no se proporciona una específica
    fecha_a_usar = fecha if fecha is not None else datetime.now()
    
    # Formatear la fecha
    ruta_fecha = fecha_a_usar.strftime(f"%Y{separador}%m{separador}%d")
    
    return ruta_fecha

In [ ]:
def obtener_columnas_destino(connection_params):
    """
    Obtiene la lista de columnas de la tabla tab_ficharecolector2
    
    Parámetros:
    - db_params: Diccionario con parámetros de conexión
        
    Retorna:
    - Lista de nombres de columnas
    """
    conexion = None
    cursor = None
    
    try:
        conexion = conectar_postgres(**connection_params)
        cursor = conexion.cursor()
        
        # Consulta para obtener columnas
        query = """
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = 'tab_ficharecolector2'
        ORDER BY ordinal_position;
        """
        
        cursor.execute(query)
        columnas = [row[0] for row in cursor.fetchall()]
        
        return columnas
        
    except Exception as e:
        print(f"Error al obtener columnas destino: {str(e)}")
        raise
        
    finally:
        if cursor:
            cursor.close()
        if conexion:
            conexion.close()

In [ ]:
def integracionpaciente(columnas_bd, connection_params):
    """
    Integra datos de pacientes desde la tabla 'paciente' a 'tab_ficharecolector2'
    
    Parámetros:
    - columnas_bd: Lista de columnas existentes en la tabla destino
    - db_params: Diccionario con parámetros de conexión a la BD:
        {
            'db_name': 'nombre_bd',
            'host': 'host_bd',
            'user': 'usuario',
            'password': 'contraseña',
            'port': 5432 (opcional)
        }
    
    Retorna:
    - True si la integración fue exitosa
    - False si hubo errores
    """
    conexion = None
    cursor = None
    
    try:
        # Establecer conexión
        conexion = conectar_postgres(**connection_params)
        cursor = conexion.cursor()
        
        # 1. Obtener datos de pacientes
        query_paciente = """
        SELECT 
            nombre1, nombre2, nombre3, 
            apellido1, apellido2, 
            sexo, id_paciente, fecha_nacimiento,
            tipo_de_identificacion, identificacion,
            municipio_residencia, barrio_residencia,
            direccion, edad
        FROM paciente;
        """
        cursor.execute(query_paciente)
        column_names = [desc[0] for desc in cursor.description]
        pacientes = cursor.fetchall()
        
        # 2. Obtener IDs existentes para evitar duplicados
        cursor.execute("SELECT llave_paciente FROM tab_ficharecolector2")
        ids_existentes = {row[0] for row in cursor.fetchall()}
        
        # 3. Preparar datos para inserción
        mapeo_campos = {
            'nombre1': 'prinom_paciente',
            'nombre2': 'segunom_paciente',
            'nombre3': 'tercernom_paciente',
            'apellido1': 'primapellido_paciente',
            'apellido2': 'seguapellido_paciente',
            'sexo': 'sexo',
            'id_paciente': 'llave_paciente',
            'fecha_nacimiento': 'fenaci_paciente',
            'tipo_de_identificacion': 'tipo_documento',
            'identificacion': 'numid_paciente',
            'municipio_residencia': 'municipio_resi1',
            'barrio_residencia': 'comunabarrio_resi1',
            'direccion': 'dir_residencia1',
            'edad': 'edad_paciente'
        }
        
        # 4. Filtrar y transformar datos
        registros_a_insertar = []
        for paciente in pacientes:
            paciente_dict = dict(zip(column_names, paciente))
            
            # Verificar si el paciente ya existe
            if paciente_dict['id_paciente'] in ids_existentes:
                continue
                
            # Mapear campos al esquema destino
            registro = {}
            for origen, destino in mapeo_campos.items():
                if destino in columnas_bd:  # Solo incluir columnas existentes
                    registro[destino] = paciente_dict.get(origen)
            
            registros_a_insertar.append(registro)
        
        # 5. Insertar registros en lote
        if registros_a_insertar:
            columns = ', '.join(registros_a_insertar[0].keys())
            placeholders = ', '.join(['%s'] * len(registros_a_insertar[0]))
            
            insert_query = f"""
            INSERT INTO tab_ficharecolector2 ({columns})
            VALUES ({placeholders})
            """
            
            # Preparar valores para inserción masiva
            values = [tuple(registro.values()) for registro in registros_a_insertar]
            
            cursor.executemany(insert_query, values)
            conexion.commit()
            
            print(f"Se insertaron {len(registros_a_insertar)} nuevos registros de pacientes")
        else:
            print("No hay nuevos registros de pacientes para insertar")
        
        return True
        
    except Exception as e:
        print(f"Error en integración de pacientes: {str(e)}")
        if conexion:
            conexion.rollback()
        return False
        
    finally:
        # Cerrar cursor y conexión
        if cursor:
            cursor.close()
        if conexion:
            conexion.close()
            print("Conexión a PostgreSQL cerrada")

In [ ]:
import pandas as pd
import numpy as np
from psycopg2.extras import RealDictCursor
from psycopg2 import OperationalError

def integracion_obs_infolab(connection_params):
    """
    Integra observaciones desde la tabla infolab hacia tab_ficharecolector2.
    Usa la función conectar_postgres con los parámetros proporcionados.
    """
    connection = None

    try:
        # Usar directamente los params recibidos
        connection = conectar_postgres(
            db_name=connection_params['db_name'],
            host=connection_params['host'],
            user=connection_params['user'],
            password=connection_params['password'],
            port=connection_params['port']
        )

        columnas_bd = obtener_columnas_destino(connection_params)
        if not columnas_bd:
            print("No se pudieron obtener las columnas destino")
            return False

        # 1. Leer tabla infolab
        query_infolab = """
        SELECT id_paciente, observacion, fecha_informe, fuente
        FROM infolab_clean
        ORDER BY id_paciente, fecha_informe;
        """
        df_infolab = pd.read_sql(query_infolab, connection)

        # 2. Leer tabla tab_ficharecolector2
        query_ficha = """
        SELECT llave_paciente, obser_fue1, obser_fue2, obser_fue3, 
               obser_fue4, obser_fue5, obser_fue6, obser_fue7, obser_fue8
        FROM tab_ficharecolector2;
        """
        df_ficha = pd.read_sql(query_ficha, connection).drop_duplicates(subset='llave_paciente')

        # 3. Agrupar y limitar por paciente
        def limitar_y_rellenar(x): return list(x)[:8] + [np.nan] * (8 - len(x[:8]))

        df_obs = df_infolab.groupby('id_paciente')['observacion'].apply(limitar_y_rellenar).apply(pd.Series)
        df_fechas = df_infolab.groupby('id_paciente')['fecha_informe'].apply(limitar_y_rellenar).apply(pd.Series)
        df_fuentes = df_infolab.groupby('id_paciente')['fuente'].apply(limitar_y_rellenar).apply(pd.Series)

        df_obs.columns = [f"obser_fue{i+1}" for i in range(8)]
        df_fechas.columns = [f"fecha_fue{i+1}" for i in range(8)]
        df_fuentes.columns = [f"fuente{i+1}" for i in range(8)]

        df_unificado = pd.concat([df_obs, df_fechas, df_fuentes], axis=1)
        df_unificado.reset_index(inplace=True)

        # 4. Merge con tabla ficharecolector
        df_update = pd.merge(
            df_ficha[['llave_paciente']],
            df_unificado,
            left_on='llave_paciente',
            right_on='id_paciente',
            how='left'
        ).drop(columns=['id_paciente'])

        # 5. Columnas destino
        columnas_similares = [
            col for col in [
                "obser_fue1", "obser_fue2", "obser_fue3", "obser_fue4",
                "obser_fue5", "obser_fue6", "obser_fue7", "obser_fue8",
                "fecha_fue1", "fecha_fue2", "fecha_fue3", "fecha_fue4",
                "fecha_fue5", "fecha_fue6", "fecha_fue7", "fecha_fue8",
                "fuente1", "fuente2", "fuente3", "fuente4",
                "fuente5", "fuente6", "fuente7", "fuente8"
            ] if col in columnas_bd
        ]

        set_clause = ', '.join([f"{col} = %({col})s" for col in columnas_similares])
        update_query = f"""
        UPDATE tab_ficharecolector2
        SET {set_clause}
        WHERE llave_paciente = %(llave_paciente)s;
        """

        # 6. Ejecutar UPDATE
        with connection.cursor() as cur:
            for _, row in df_update.iterrows():
                update_data = {col: row[col] for col in columnas_similares + ['llave_paciente']}
                cur.execute(update_query, update_data)
            connection.commit()

        print("Actualización completada con éxito.")
        return True

    except Exception as e:
        if connection:
            connection.rollback()
        print(f"Error durante la integración: {e}")
        raise

    finally:
        if connection:
            connection.close()
            print("Conexión cerrada.")

In [ ]:
import pandas as pd
import numpy as np
from psycopg2.extras import RealDictCursor
from psycopg2 import OperationalError

def integracion_obs_infolab_difuntos(connection_params):
    """
    Integra observaciones desde la tabla infolab hacia tab_ficharecolector2.
    Usa la función conectar_postgres con los parámetros proporcionados.
    """
    connection = None

    try:
        # Usar directamente los params recibidos
        connection = conectar_postgres(
            db_name=connection_params['db_name'],
            host=connection_params['host'],
            user=connection_params['user'],
            password=connection_params['password'],
            port=connection_params['port']
        )

        columnas_bd = obtener_columnas_destino(connection_params)
        if not columnas_bd:
            print("No se pudieron obtener las columnas destino")
            return False

        # 1. Leer tabla infolab
        query_infolab = """
        SELECT id_paciente, observaciones_fuente observacion, fecha_informe, 'DIF' fuente
        FROM infolab_difuntos_clean
        ORDER BY id_paciente;
        """
        df_infolab = pd.read_sql(query_infolab, connection)

        # 2. Leer tabla tab_ficharecolector2
        query_ficha = """
        SELECT llave_paciente, obser_fue1, obser_fue2, obser_fue3, 
               obser_fue4, obser_fue5, obser_fue6, obser_fue7, obser_fue8
        FROM tab_ficharecolector2;
        """
        df_ficha = pd.read_sql(query_ficha, connection).drop_duplicates(subset='llave_paciente')

        # 3. Agrupar y limitar por paciente
        def limitar_y_rellenar(x): return list(x)[:8] + [np.nan] * (8 - len(x[:8]))

        df_obs = df_infolab.groupby('id_paciente')['observacion'].apply(limitar_y_rellenar).apply(pd.Series)
        df_fechas = df_infolab.groupby('id_paciente')['fecha_informe'].apply(limitar_y_rellenar).apply(pd.Series)
        df_fuentes = df_infolab.groupby('id_paciente')['fuente'].apply(limitar_y_rellenar).apply(pd.Series)

        df_obs.columns = [f"obser_fue{i+1}" for i in range(8)]
        df_fechas.columns = [f"fecha_fue{i+1}" for i in range(8)]
        df_fuentes.columns = [f"fuente{i+1}" for i in range(8)]

        df_unificado = pd.concat([df_obs, df_fechas, df_fuentes], axis=1)
        df_unificado.reset_index(inplace=True)

        # 4. Merge con tabla ficharecolector
        df_update = pd.merge(
            df_ficha[['llave_paciente']],
            df_unificado,
            left_on='llave_paciente',
            right_on='id_paciente',
            how='left'
        ).drop(columns=['id_paciente'])

        # 5. Columnas destino
        columnas_similares = [
            col for col in [
                "obser_fue1", "obser_fue2", "obser_fue3", "obser_fue4",
                "obser_fue5", "obser_fue6", "obser_fue7", "obser_fue8",
                "fecha_fue1", "fecha_fue2", "fecha_fue3", "fecha_fue4",
                "fecha_fue5", "fecha_fue6", "fecha_fue7", "fecha_fue8",
                "fuente1", "fuente2", "fuente3", "fuente4",
                "fuente5", "fuente6", "fuente7", "fuente8"
            ] if col in columnas_bd
        ]

        set_clause = ', '.join([f"{col} = %({col})s" for col in columnas_similares])
        update_query = f"""
        UPDATE tab_ficharecolector2
        SET {set_clause}
        WHERE llave_paciente = %(llave_paciente)s;
        """

        # 6. Ejecutar UPDATE
        with connection.cursor() as cur:
            for _, row in df_update.iterrows():
                update_data = {col: row[col] for col in columnas_similares + ['llave_paciente']}
                cur.execute(update_query, update_data)
            connection.commit()

        print("Actualización completada con éxito.")
        return True

    except Exception as e:
        if connection:
            connection.rollback()
        print(f"Error durante la integración: {e}")
        raise

    finally:
        if connection:
            connection.close()
            print("Conexión cerrada.")

In [ ]:
import pandas as pd
import numpy as np
from psycopg2.extras import RealDictCursor
from psycopg2 import OperationalError

def crear_df_ficharecolector(connection_params):
    """
    Integra observaciones desde la tabla infolab hacia tab_ficharecolector2.
    Usa la función conectar_postgres con los parámetros proporcionados.
    """
    connection = None

    try:
        # Usar directamente los params recibidos
        connection = conectar_postgres(
            db_name=connection_params['db_name'],
            host=connection_params['host'],
            user=connection_params['user'],
            password=connection_params['password'],
            port=connection_params['port']
        )

        # 1. Leer tabla infolab
        query = """
        SELECT *
        FROM tab_ficharecolector2
        WHERE llave_paciente IS NOT NULL;
        """

        df= pd.read_sql(query, connection)
        if df.empty:
            print("No se encontraron registros en tab_ficharecolector2.")
            return None
        return df
    except OperationalError as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

In [ ]:
#import pandas as pd
#print(pd.__version__)